<a href="https://colab.research.google.com/github/AdarshKhatri01/DBMS-Notes/blob/main/Parallel_Distributed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CHAPTER-21**

### **1. What is Range Partitioning?**

Range partitioning is a way to organize data across multiple disks based on ranges of values. For example, if you have a database of employees, you might store employees with IDs 1–1000 on Disk 1, IDs 1001–2000 on Disk 2, and so on.

### **The Property in Question**
In range partitioning, when you query a specific range of data (e.g., "find all employees with IDs between 500 and 600"), it's possible that only **one disk** will need to be accessed because the entire range of data you're looking for might be stored on just one disk.

---

### **Benefits of This Property**
1. **Fast Query Execution for Small Ranges:**
   - If the range you're querying is small (e.g., only 100 employees), and all those employees are stored on one disk, the query can be processed quickly.
   - You don’t waste time accessing multiple disks, which reduces overhead and speeds up the process.

2. **Efficient Use of Resources:**
   - Since only one disk is involved, other disks remain free to handle different queries at the same time. This allows for better parallel execution of multiple queries.

---

### **Drawbacks of This Property**
1. **Slow Query Execution for Large Ranges:**
   - If the range you're querying is large (e.g., "find all employees with IDs between 1 and 10,000"), and all those employees are stored on just one disk, the query will take a long time to execute.
   - There’s no parallelism within the query because only one disk is doing all the work.

2. **Hot Spots:**
   - Certain disks might become "hot spots" if they are frequently accessed for queries involving specific ranges. This can overload those disks, increasing response times and potentially causing bottlenecks.

---

### **Solution: Hybrid Range Partitioning**
To address these drawbacks, a technique called **hybrid range partitioning** can be used. In this approach:
- Data is divided into smaller ranges (e.g., each range contains only a few blocks of data).
- These smaller ranges are distributed across disks in a **round-robin fashion**, meaning no single disk gets all the data for a large range.

This provides the benefits of range partitioning (efficient access for small ranges) while avoiding its drawbacks (slow performance for large ranges and hot spots).

---

### **Summary**
- **Benefit:** Queries for small ranges are fast because they only need to access one disk.
- **Drawback:** Queries for large ranges can be slow because there’s no parallelism, and some disks may become overloaded.
- **Solution:** Use hybrid range partitioning to balance the load and improve performance for both small and large queries.

By using hybrid range partitioning, you get the best of both worlds: efficient queries for small ranges and better performance for large ranges.

**Final Answer:**
$$
\boxed{\text{Hybrid range partitioning balances the benefits and drawbacks of range partitioning by distributing small ranges across disks in a round-robin manner.}}
$$

---
---
---

<br/>

---
---
---

### **Why are traditional histograms not useful for avoiding execution skew?**

Histograms are statistical representations of how data is distributed across a relation (table). They divide the range of attribute values into buckets and show how many tuples (rows) fall into each bucket. This helps in understanding **data distribution skew**, i.e., whether some ranges of values have significantly more or fewer tuples than others.

However, **execution skew** refers to uneven workloads across processing units (e.g., disks or processors) during query execution. Traditional histograms focus only on the data distribution and do not account for:
- How frequently specific values are queried.
- How queries are executed (e.g., point lookups vs. range queries).
- The actual workload characteristics.

For example:
- If certain values are queried much more frequently than others, even if the data is evenly distributed, the processing units responsible for those values will become overloaded, leading to execution skew.
- Histograms alone cannot capture this information about query frequency or access patterns, which is why they are not effective at avoiding execution skew.

---

#### **2. How can you use query workloads to design a partitioning scheme that avoids execution skew?**

To avoid execution skew, we need to consider not just the data distribution but also the **query workload**. Specifically, for a workload of **point lookup queries** (queries that retrieve rows based on exact matches of attribute values), we can use the following steps:

---

### **Step-by-Step Explanation**

#### **Step 1: Analyze the Query Workload**
- Collect statistics about the workload, such as:
  - Which attribute values are most frequently queried.
  - How often each value appears in the queries.
- For example, if you have a table of employees and queries frequently look up employees with IDs `101`, `205`, and `307`, these IDs are "hot" values.

#### **Step 2: Identify Hot and Cold Values**
- Divide the attribute values into two categories:
  - **Hot values:** Frequently accessed values (e.g., `101`, `205`, `307`).
  - **Cold values:** Infrequently accessed values.
- The goal is to distribute the hot values across different partitions to balance the workload.

#### **Step 3: Design a Partitioning Scheme**
- Use the query workload statistics to create a **partitioning scheme** that balances the query load:
  - **Round-robin partitioning for hot values:** Distribute hot values evenly across partitions. For example, assign `101` to Disk 1, `205` to Disk 2, and `307` to Disk 3.
  - **Range or hash partitioning for cold values:** Use traditional range or hash partitioning for cold values, as they do not contribute significantly to execution skew.
- This ensures that no single partition becomes a bottleneck due to frequent queries.

#### **Step 4: Monitor and Adjust**
- Continuously monitor the query workload and adjust the partitioning scheme as needed. Query patterns may change over time, so the partitioning scheme should be dynamic and adaptive.

---

### **Why This Works**
By considering both the **data distribution** (via histograms) and the **query workload** (via query statistics), we can design a partitioning scheme that minimizes execution skew. Specifically:
- Frequent queries are distributed across multiple partitions, ensuring balanced workload.
- Infrequent queries do not overload any single partition.

---

### **Final Answer**
$$
\boxed{\text{To avoid execution skew in a workload of point lookup queries, analyze query frequencies to identify hot and cold values, then design a partitioning scheme that distributes hot values evenly across partitions while using range or hash partitioning for cold values.}}
$$

---
---
---

<br/>

---
---
---


#### **a. Why replicate data across geographically distributed data centers?**

1. **Disaster Recovery and High Availability:**
   - If one data center fails (e.g., due to a power outage, fire, or natural disaster like an earthquake), the data is still available in another data center. This ensures that your system keeps running without interruptions.
   - By placing data centers far apart geographically, you reduce the risk of both data centers being affected by the same disaster at the same time.

2. **Faster Access for Users in Different Locations:**
   - If users are spread across different regions, having data replicated in multiple locations allows them to access the data from a nearby data center. This reduces delays (latency) and improves performance.

---

#### **b. How is replication in centralized databases different from parallel/distributed databases?**

1. **Centralized Databases:**
   - In centralized databases, replication typically involves copying the **entire database** to another location using **log records** (which track changes made to the database).
   - Some centralized databases allow partial replication (only certain tables or parts of the database), but this is less common.
   - Centralized databases do not support partitioning (splitting the database into smaller parts). If one node fails, the entire load shifts to the other node, which can overload it.

2. **Parallel/Distributed Databases:**
   - In parallel/distributed databases, the database is often split into smaller parts (partitioned), and each part can be replicated independently on different nodes.
   - This means that when one node fails, only the specific partitions stored on that node need to be handled by other nodes. The load is spread out, reducing the risk of overloading any single node.
   - Distributed databases are designed to handle failures more efficiently because they distribute both the data and the processing workload across multiple nodes.

---

### **Key Differences Summarized**

| **Aspect**                     | **Centralized Databases**                          | **Parallel/Distributed Databases**               |
|---------------------------------|---------------------------------------------------|------------------------------------------------|
| **Replication Scope**           | Entire database is replicated                    | Specific parts (partitions) of the database can be replicated independently |
| **Partitioning Support**        | No support for partitioning                      | Supports partitioning to distribute workload  |
| **Failure Handling**            | Overloads a single replica if a node fails       | Spreads the load across multiple nodes        |
| **Use Case**                    | Simpler systems with limited scalability         | Designed for large-scale, fault-tolerant systems |

---

### **Final Answer**
$$
\boxed{
\text{a. Replicating data across geographically distributed data centers ensures high availability (disaster recovery) and faster access for users in different regions.}
}
$$
$$
\boxed{
\text{b. Centralized databases replicate the entire database using log records, while distributed databases support partitioning and independent replication of parts, spreading the load more efficiently.}
}
$$

---
---
---

<br/>

---
---
---

#### **a. Why would storing a partition number and record identifier in a global secondary index be a bad idea?**

In a distributed database, data is split into partitions, and these partitions can be moved or split to balance the load across nodes. If a global secondary index stores both the **partition number** and the **record identifier**, every time a partition is moved or split:
- The secondary index would need to be updated for **every single record** in that partition.
- This could result in a **huge number of updates**, which would slow down the system and make it inefficient.

For example:
- Imagine you have 1 million records in a partition, and you decide to move that partition to another node. You would need to update the secondary index for all 1 million records to reflect the new partition number. This is computationally expensive and impractical.

Instead, a better approach is to use an **indirect reference** (like a clustering key or partitioning key) that doesn’t change when partitions are moved or split. This avoids the need for frequent updates to the secondary index.

---

#### **b. How are global secondary indexes similar to local secondary indexes in B+ tree file organizations?**

Both **global secondary indexes** (in distributed databases) and **local secondary indexes** (in centralized B+ tree file organizations) face a similar challenge: **records can move**. Here’s why this leads to a similar implementation:

1. **Records Move in Both Scenarios:**
   - In **distributed databases**, records may move between nodes (e.g., due to partition splitting or load balancing).
   - In **B+ tree file organizations**, records may move within the same node (e.g., due to changes in the B+ tree structure, like splitting or merging nodes).

2. **Direct Pointers Are Problematic:**
   - If secondary indexes stored **direct pointers** (e.g., exact memory addresses or specific partition numbers), they would need to be updated every time a record moves.
   - For example:
     - In a distributed database, if a record moves to a new partition, the secondary index would need to update the partition number.
     - In a B+ tree, if a record moves to a new location in the tree, the secondary index would need to update the pointer to its new location.

3. **Indirection Solves the Problem:**
   - Instead of storing direct pointers, both types of secondary indexes use an **indirect reference**:
     - In **distributed databases**, the secondary index uses a **partitioning key** or **clustering key** to locate the record, rather than directly storing the partition number.
     - In **B+ tree file organizations**, the secondary index uses the **primary key** (or clustering key) to locate the record, rather than storing its exact location in the tree.
   - This indirection means that even if the record moves, the secondary index doesn’t need to be updated because the key remains the same.

---

### **Key Similarities Between Global and Local Secondary Indexes**

| **Aspect**                     | **Global Secondary Index (Distributed DB)**         | **Local Secondary Index (B+ Tree)**              |
|---------------------------------|----------------------------------------------------|------------------------------------------------|
| **Record Movement**             | Records move across nodes/partitions              | Records move within the B+ tree structure      |
| **Problem with Direct Pointers**| Frequent updates needed if partition numbers change| Frequent updates needed if locations change    |
| **Solution**                    | Use indirect references (partitioning/clustering keys)| Use indirect references (primary/clustering keys)|
| **Advantage of Indirection**    | Avoids updating the index when records move        | Avoids updating the index when records move    |

---

### **Final Answer**

$$
\boxed{
\text{a. Storing partition numbers in a global secondary index would require frequent updates whenever partitions are moved or split, making it inefficient.}
}
$$
$$
\boxed{
\text{b. Both global and local secondary indexes avoid frequent updates by using indirect references (keys) instead of direct pointers, allowing records to move without affecting the index.}
}
$$

---
---
---

<br/>

---
---
---

#### **a. Why distribute copies of data items across multiple nodes instead of storing all copies in the same node?**

1. **Better Workload Distribution During Failures:**
   - If one node fails, its workload (e.g., processing queries) needs to be handled by other nodes.
   - If all copies of the failed node's data are stored on just one or a few other nodes, those nodes will become overloaded.
   - By distributing the copies across **multiple nodes**, the workload can be spread out evenly, reducing the risk of overloading any single node.

2. **Handling Hotspots for Read-Only Queries:**
   - Sometimes, certain data items are accessed very frequently (called "hotspots").
   - If copies of these hot data items are spread across multiple nodes, read-only queries can be distributed among those nodes, improving performance and avoiding bottlenecks.

---

#### **b. What are the benefits and drawbacks of using RAID storage instead of storing an extra copy of each data item manually?**

**RAID (Redundant Array of Independent Disks)** is a technology that combines multiple physical disks into a single logical unit to improve performance and/or reliability. Let’s compare it with manually storing an extra copy of each data item:

---

### **Benefits of RAID Storage:**

1. **Automatic Mirroring (RAID Level 0):**
   - In RAID Level 0 (mirroring), the system automatically creates a duplicate copy of each data item on another disk.
   - The database system doesn’t need to manage this mirroring process—it simply writes data to the RAID system, and RAID takes care of duplicating it.
   - This simplifies the work for the database system.

2. **Cost Efficiency (RAID Level 5):**
   - RAID Level 5 uses a technique called **parity** to store redundant information without duplicating every single data item. This requires less disk space compared to manually storing an extra copy of everything.
   - For example, instead of needing double the storage space (like manual duplication), RAID Level 5 might only require 20–30% extra space.

---

### **Drawbacks of RAID Storage:**

1. **Expensive Writes (RAID Level 5):**
   - In RAID Level 5, every write operation involves calculating and updating parity information, which makes writes slower and more expensive compared to simple duplication.

2. **Rebuilding After Disk Failure (RAID Level 5):**
   - If a disk in a RAID Level 5 array crashes, rebuilding the lost data from the remaining disks is computationally expensive and time-consuming.
   - In contrast, if you manually store an extra copy of each data item, rebuilding is simpler because you already have a complete duplicate.

3. **No Control Over Placement:**
   - With RAID, the database system has no control over how data is distributed across disks. This can lead to inefficiencies if certain disks become overloaded or develop hotspots.
   - Manual duplication allows the database system to decide where to place copies, potentially optimizing performance.

---

### **Key Comparison Between RAID and Manual Duplication**

| **Aspect**                     | **RAID Storage**                                  | **Manual Duplication**                          |
|---------------------------------|--------------------------------------------------|-----------------------------------------------|
| **Ease of Management**          | Automatic mirroring (simpler for the database)   | Requires manual management                   |
| **Disk Space Usage**            | RAID Level 5 is more space-efficient             | Requires double the storage space            |
| **Write Performance**           | Slower writes (due to parity calculations)       | Faster writes (no parity overhead)           |
| **Rebuilding After Failure**    | Expensive and slow (RAID Level 5)                | Simpler and faster                           |
| **Workload Distribution**       | No control over placement, may create hotspots  | Can optimize placement to avoid hotspots     |

---

### **Final Answer**

$$
\boxed{
\text{a. Distributing copies of data items across multiple nodes helps balance the workload during failures and reduces hotspots caused by frequent read-only queries.}
}
$$
$$
\boxed{
\text{b. RAID storage simplifies mirroring and can save disk space (especially RAID Level 5), but it has slower writes and more expensive recovery after disk failures compared to manual duplication.}
}
$$

---
---
---

<br/>

---
---
---

### **Simple Explanation**

---

#### **a. Why does range-partitioning give better control on tablet sizes than hash partitioning?**

1. **Range Partitioning:**
   - In range partitioning, data is divided into ranges based on the values of a key (e.g., IDs 1–1000, 1001–2000, etc.).
   - If one range becomes too large (overfull), it can easily be split into smaller ranges (e.g., splitting 1–1000 into 1–500 and 501–1000). This gives fine-grained control over tablet sizes.
   - For example, in a B+ tree index, if a leaf node becomes overfull, it can be split into two nodes.

2. **Hash Partitioning:**
   - In hash partitioning, data is distributed across partitions based on a hash function applied to the key.
   - If one partition becomes overfull, it’s harder to fix because you can’t just split it like in range partitioning. You would need to rehash all the data or use advanced techniques like dynamic hashing (e.g., linear hashing or extendable hashing), which are more complex.

**Analogy with B+ Tree vs. Hash Index:**
- A **B+ tree index** (range-based) allows easy splitting of overfull nodes, similar to how range partitioning allows splitting of overfull tablets.
- A **hash index** doesn’t allow easy splitting of overfull buckets, similar to how hash partitioning struggles with overfull partitions.

---

#### **b. Why might some systems first hash keys and then perform range partitioning on the hash values? What are the drawbacks compared to direct range partitioning?**

1. **Motivation for Hashing + Range Partitioning:**
   - **Simplifies Partitioning:** Hashing converts keys of various types (e.g., strings, numbers) into a single numeric type (hash values), making it easier to partition the data.
   - **Balanced Distribution:** Hashing ensures that data is evenly distributed across partitions, avoiding hotspots caused by uneven data distribution.

2. **Drawbacks Compared to Direct Range Partitioning:**
   - **No Support for Range Queries:** With hashing, range queries (e.g., "find all records with IDs between 100 and 200") cannot be supported efficiently because the original order of the keys is lost. To answer such queries, you might need to scan the entire dataset.
   - **Direct Range Partitioning Advantage:** Direct range partitioning preserves the order of keys, allowing efficient support for range queries without requiring a full table scan.

---

#### **c. What are the benefits of horizontally partitioning data first and then performing vertical partitioning locally at each node, compared to vertically partitioning first and then horizontally partitioning independently?**

1. **First Option (Horizontal → Vertical):**
   - After horizontal partitioning, each node contains a subset of the rows (records).
   - Then, vertical partitioning splits the columns (attributes) within each node.
   - **Benefit:** If a query only accesses records at a single node, the entire record can be reconstructed locally at that node without needing communication with other nodes.

2. **Second Option (Vertical → Horizontal):**
   - After vertical partitioning, each node contains a subset of the columns (attributes).
   - Then, horizontal partitioning splits the rows (records) independently for each column subset.
   - **Drawback:** The vertical fragments corresponding to the same record may end up on different nodes. To reconstruct a complete record, extra communication is required between nodes, which increases overhead.

---

### **Final Answer**

$$
\boxed{
\text{a. Range partitioning gives better control over tablet sizes because overfull partitions can be easily split, unlike hash partitioning. This is analogous to how B+ tree indexes allow splitting overfull nodes, while hash indexes do not.}
}
$$

$$
\boxed{
\text{b. Hashing simplifies partitioning and balances data distribution but sacrifices efficient range query support. Direct range partitioning preserves key order, enabling efficient range queries.}
}
$$

$$
\boxed{
\text{c. Horizontally partitioning first and then vertically partitioning locally allows records to be reconstructed at a single node, avoiding extra communication. Vertically partitioning first requires extra communication to gather vertical fragments from different nodes.}
}
$$

---
---
---

<br/>

---
---
---


#### **a. What happens if the master replica changes between identifying it and sending a request? How can this be handled?**

- In distributed systems, the "master replica" is the node responsible for handling updates to a specific data item.
- If a node identifies the master replica but, by the time the request reaches that node, the mastership has changed (e.g., due to a failure or load balancing), the situation needs to be resolved.

**How to Handle This:**
1. **Error Reply:**
   - The old master (the node that received the request) sends an error reply back to the requesting node, saying, "I’m no longer the master."
   - The requesting node then looks up the current master and resends the request to the correct node.

2. **Forwarding the Request:**
   - Instead of replying with an error, the old master forwards the request directly to the new master.
   - The new master processes the request and replies to the requesting node.

Both approaches ensure the request eventually reaches the correct master replica.

---

#### **b. What are the benefits of tracking the master replica on a per-record basis instead of a per-partition basis?**

In some systems, each record in a partition can have its own master replica, independent of other records. This approach has two key benefits:

1. **Geographical Optimization:**
   - The master replica for a record can be placed in the geographical region where most requests for that record occur.
   - For example, if most users accessing a record live in Europe, the master replica for that record can be located in a European data center. This reduces delays caused by communication across long distances (speed-of-light delays).

2. **Faster Reads and Writes:**
   - **Reads:** Since the master replica is closer to the user, read operations can be performed locally without needing to contact other regions.
   - **Writes:** Write operations can also happen locally at the master replica and then be asynchronously replicated to other replicas. This improves performance and reduces latency.

---

#### **c. How can we track the master replica for each record when there are a large number of records?**

Tracking the master replica for every record in a system with millions (or billions) of records requires efficient mechanisms. Here’s how it can be done:

1. **Hidden Field in Each Record:**
   - Each record can include a hidden field that stores the identifier (e.g., node address) of its master replica.
   - When a request is made for a record, the system first checks this hidden field to find the master replica.

2. **Handling Outdated Information:**
   - If the hidden field contains outdated information (e.g., the master replica has changed), the system can:
     - Access all replicas of the record to determine which node is currently listed as the master.
     - Contact those nodes to identify the current master replica.

3. **Centralized Metadata Store (Optional):**
   - Alternatively, a centralized metadata store can maintain a mapping of records to their master replicas. When a request is made, the system queries this metadata store to find the correct master replica.

This approach ensures that even with a large number of records, the system can efficiently track and locate the master replica for any given record.

---

### **Final Answer**

$$
\boxed{
\text{a. If the master replica changes after identification, the old master can either send an error reply or forward the request to the new master.}
}
$$

$$
\boxed{
\text{b. Tracking mastership on a per-record basis allows geographically optimized placement of masters (reducing delays) and faster local reads/writes.}
}
$$

$$
\boxed{
\text{c. Use a hidden field in each record to store the master replica's identifier. If outdated, check all replicas or use a centralized metadata store to find the current master.}
}
$$